<a href="https://colab.research.google.com/github/0xalphaprime/ai-ml-data/blob/main/NLP_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NER is a token classification
# IOB tagging format


In [ ]:
# set up the local env


!pip install -U bitsandbytes
!pip install -U torch
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn

## What is NER

NER stands for **Named Entity Recognition**, which is a natural language processing technique used to identify and classify named entities (such as names of people, places, organizations, dates, and more) in text. Here's a breakdown of NER in five steps:

**Text Input:** NER begins with a piece of text, which can be a sentence, a paragraph, a document, or even a larger corpus of text.

**Tokenization:** The input text is split into individual words or tokens. This process is called tokenization, and it's a crucial step because NER works on a token-by-token basis.

**Entity Recognition:**
The tokenized text is analyzed to identify spans of tokens that correspond to named entities. NER systems use various techniques, such as rule-based approaches, machine learning models (like conditional random fields or deep learning models), or a combination of these methods to recognize entities.

**Entity/Token Classification:** Once the entities are recognized, they are classified into predefined categories like "person," "organization," "location," "date," "number," etc. These categories help organize and provide context to the recognized entities.

**Output:** The final output of the NER process is a structured representation of the original text with identified named entities and their corresponding categories. This output can be used for various purposes like information extraction, content summarization, sentiment analysis, and more.


+++++++++++++++++++

IOB tagging, which stands for Inside-Outside-Beginning tagging, is a common technique used in named entity recognition (NER) to label tokens in a text sequence to indicate their positions within named entities. It helps identify the starting, inside, and outside parts of entities within the text.

+++++++++++++++++++

The Transformer architecture's key innovation is the mechanism of self-attention, which allows the model to weigh the importance of different words in a sentence relative to each other. This self-attention mechanism enables the model to capture contextual relationships between words regardless of their position in the input sequence.

## The Dataset

###CONLLPP Dataset

[HuggingFace Dataset Link](https://huggingface.co/datasets/conllpp)

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
data = load_dataset("conllpp")
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
data['train'].features


{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [ ]:
# converting to a panda dataframe to get to know the data better

pd.DataFrame(data['train'][:])[['tokens', 'ner_tags']].iloc[0]

tokens      [EU, rejects, German, call, to, boycott, Briti...
ner_tags                          [3, 0, 7, 0, 0, 0, 7, 0, 0]
Name: 0, dtype: object

In [ ]:
tags = data['train'].features['ner_tags'].feature

index2tag = {idx:tag for idx, tag in enumerate(tags.names)}
tag2index = {tag:idx for idx, tag in enumerate(tags.names)}


In [ ]:
index2tag


{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [ ]:
tags.int2str(2)


'I-PER'

In [ ]:
def create_tag_names(batch):
  tag_name = { 'ner_tags_str': [tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name


In [ ]:
data = data.map(create_tag_names)
data


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3453
    })
})

In [ ]:
pd.DataFrame(data['train'][:])[['tokens', 'ner_tags', 'ner_tags_str']].iloc[0]

# notice the ner_tag_str now visible in the data structure

tokens          [EU, rejects, German, call, to, boycott, Briti...
ner_tags                              [3, 0, 7, 0, 0, 0, 7, 0, 0]
ner_tags_str            [B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]
Name: 0, dtype: object

## Building the Model

In [ ]:
# Tokenization

from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True